# Homework 3

In [7]:
# Import Packages
import numpy as np

In [3]:
def readVectorsSeq(filename):
    
    return

In [4]:
def kcenter(P,k):
    return

In [5]:
def kmeansPP(P,k):
    return

In [6]:
def kmeansObj(P,C):
    return